In [1]:
import os
import warnings
import sys
from datetime import datetime

In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

In [4]:
from urllib.parse import urlparse

In [5]:
import mlflow
import mlflow.sklearn

In [6]:
import logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [7]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)

    # Log a metric; metrics can be updated throughout the run
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2", r2)

    print("RMSE {}".format(rmse))
    print("MAE {}".format(mae))
    print("R2 {}".format(r2))
    
    return rmse, mae, r2

In [8]:
def download_dataset():
    import urllib.request
    import zipfile
    
    url = "https://archive.ics.uci.edu/static/public/186/wine+quality.zip"
    filename = '.././data/wine.zip'
    urllib.request.urlretrieve(url, filename)
    
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall('.././data/')


In [9]:
def log_params(params):
    print("Params")    
    for key, value in params.items():        
        # Log a parameter (key-value pair)
        mlflow.log_param(key, value)
        print("{}={}".format(key, value))

In [10]:
if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    download_dataset()

    try:
        data = pd.read_csv(".././data/winequality-red.csv", sep=";")
    except Exception as e:
        logger.exception(str(e))

    train, test = train_test_split(data)
    train_X = train.drop(["quality"], axis=1)
    train_y = train[["quality"]]

    test_X = test.drop(["quality"], axis=1)
    test_y = test[["quality"]]

    params = {
        "alpha": 0.5,
        "l1_ratio": 0.5,
        "random_state": 42
    }
    
    mlflow.set_tracking_uri("http://localhost:5000")
    mlflow.set_experiment("my-experiment")
    with mlflow.start_run(run_name="run_{}".format(datetime.now())):        
        lr = ElasticNet(alpha = params["alpha"], 
                        l1_ratio = params["l1_ratio"], 
                        random_state=params["random_state"])
        lr.fit(train_X, train_y)
        
        predicted_qualities = lr.predict(test_X)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print(mlflow.get_tracking_uri())
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        print(tracking_url_type_store)
        
        # Log a parameter (key-value pair)
        log_params(params)


        if tracking_url_type_store == "file":
            mlflow.sklearn.log_model(lr, "model", registered_model_name="ElasticWineModel")
        else:
            mlflow.sklearn.log_model(lr, "model")
            


RMSE 0.7931640229276851
MAE 0.6271946374319586
R2 0.10862644997792614
http://localhost:5000
http
Params
alpha=0.5
l1_ratio=0.5
random_state=42
